In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate -U

In [ ]:
# /content/drive/MyDrive/NLP and Gen-AI 13 Jan 24  /Daily Class Notes/24_02_27_Project/complaints.csv

In [ ]:
#Difference between transfer learning vs fine-tuning

In [ ]:
##Problem understanding :
business point of view:
sorting complents according to department, which is responsible for that complaint

in technical point of view:
multiclass classification
texual data

In [ ]:
##Data gathering:

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/NLP and Gen-AI 13 Jan 24  /Daily Class Notes/24_02_27_Project/complaints.csv")

In [4]:
df.head(5)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-06-13,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,CAPITAL ONE FINANCIAL CORPORATION,PA,186XX,NaN,Consent not provided,Web,2019-06-13,Closed with explanation,Yes,NaN,3274605
1,2019-11-01,Vehicle loan or lease,Loan,Struggling to pay your loan,Denied request to lower payments,I contacted Ally on Friday XX/XX/XXXX after fa...,Company has responded to the consumer and the ...,ALLY FINANCIAL INC.,NJ,088XX,NaN,Consent provided,Web,2019-11-01,Closed with explanation,Yes,NaN,3425257
2,2019-04-01,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,19067,NaN,Consent not provided,Web,2019-04-01,Closed with explanation,Yes,NaN,3198225
3,2021-11-01,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",GA,31707,NaN,NaN,Web,2021-11-01,In progress,Yes,NaN,4863965
4,2021-11-02,Debt collection,Medical debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Medical Data Systems, Inc.",VA,22033,NaN,NaN,Web,2021-11-02,In progress,Yes,NaN,4866449


In [5]:
df = df[["Product", "Consumer complaint narrative"]]

In [6]:
df.head(5)

,Product,Consumer complaint narrative
0,"Credit reporting, credit repair services, or o...",NaN
1,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
2,"Credit reporting, credit repair services, or o...",NaN
3,"Credit reporting, credit repair services, or o...",NaN
4,Debt collection,NaN


In [7]:
df.shape

(2326246, 2)

In [8]:
df.isna().sum()/len(df)

Product                         0.000000
Consumer complaint narrative    0.652082
dtype: float64

In [8]:
##Handling Null values:

In [9]:
df = df.dropna()

In [10]:
df.isna().sum()

Product                         0
Consumer complaint narrative    0
dtype: int64

In [11]:
df.shape

(809343, 2)

In [12]:
df["Product"].unique()

array(['Vehicle loan or lease',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Credit card or prepaid card',
       'Money transfer, virtual currency, or money service', 'Mortgage',
       'Payday loan, title loan, or personal loan', 'Debt collection',
       'Checking or savings account', 'Student loan', 'Consumer Loan',
       'Money transfers', 'Credit card', 'Bank account or service',
       'Credit reporting', 'Prepaid card', 'Payday loan',
       'Other financial service', 'Virtual currency'], dtype=object)

In [13]:
df["Product"].nunique()

18

In [14]:
df["Product"].value_counts()/len(df["Product"])*100

Credit reporting, credit repair services, or other personal consumer reports    39.101469
Debt collection                                                                 19.445526
Mortgage                                                                        10.435749
Credit card or prepaid card                                                      7.506582
Checking or savings account                                                      4.619673
Credit reporting                                                                 3.902919
Student loan                                                                     3.594768
Credit card                                                                      2.327567
Money transfer, virtual currency, or money service                               2.296060
Vehicle loan or lease                                                            1.857556
Bank account or service                                                          1.839146
Payday loa

In [ ]:
##merging classes : after discussion with client

In [15]:
class_dict =  {

'Vehicle loan or lease' : 'loan',
'Credit reporting, credit repair services, or other personal consumer reports' : 'credit_report',
'Credit card or prepaid card' : 'card' ,
'Money transfer, virtual currency, or money service' : 'money_transfer',
'Mortgage' : 'Mortgage',
'Payday loan, title loan, or personal loan' : 'loan' ,
'Debt collection' : 'Debt collection',
'Checking or savings account' : 'account',
'Student loan' : 'loan',
'Consumer Loan' : 'loan',
'Money transfers' : 'money_transfer',
'Credit card' : 'card',
'Bank account or service' : 'account',
'Credit reporting' : 'credit_report',
'Prepaid card' : 'card',
'Payday loan' : 'loan',
'Other financial service' : 'credit_report',
'Virtual currency' : 'money_transfer'
}

In [16]:
df.replace({'Product': class_dict}, inplace=True)

In [17]:
df

,Product,Consumer complaint narrative
1,loan,I contacted Ally on Friday XX/XX/XXXX after fa...
7,credit_report,Hello This complaint is against the three cred...
8,credit_report,I am a victim of Identity Theft & currently ha...
10,credit_report,Two accounts are still on my credit history af...
13,credit_report,Receiving daily telephone call ( s ) from XXXX...
...,...,...
2326240,card,"Automated calls from "" XXXX with Capital One '..."
2326241,Debt collection,I have disputed my debts several times with no...
2326242,Mortgage,My father died in XX/XX/XXXX. Left me his only...
2326243,credit_report,cfbp i would Like to file a complaint on Exper...


In [18]:
df['Product'].unique()

array(['loan', 'credit_report', 'card', 'money_transfer', 'Mortgage',
       'Debt collection', 'account'], dtype=object)

In [19]:
df['Product'].nunique()

7

In [20]:
df["Product"].value_counts()/len(df["Product"])*100

credit_report      43.040466
Debt collection    19.445526
Mortgage           10.435749
card               10.013307
loan                8.123132
account             6.458819
money_transfer      2.483002
Name: Product, dtype: float64

In [21]:
df["Product"].value_counts()

credit_report      348345
Debt collection    157381
Mortgage            84461
card                81042
loan                65744
account             52274
money_transfer      20096
Name: Product, dtype: int64

In [22]:
#data in imbalaced : undersampling

In [23]:
sampled_df = pd.DataFrame()

In [24]:
for col in df['Product'].unique():
  sample = df[df['Product'] == col].sample(1000)

  sampled_df = sampled_df.append(sample, ignore_index=True)
  # print(col)

<ipython-input-24-8cb05ae9dd0f>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sample, ignore_index=True)


In [25]:
sampled_df["Product"].value_counts()

loan               1000
credit_report      1000
card               1000
money_transfer     1000
Mortgage           1000
Debt collection    1000
account            1000
Name: Product, dtype: int64

In [26]:
sampled_df.head(5)

,Product,Consumer complaint narrative
0,loan,I signed a contract with Equitable acceptance ...
1,loan,"In XXXX, I started the application process for..."
2,loan,I went to pay on my load and as I was there I ...
3,loan,I have been enrolled in the Public Student Loa...
4,loan,the company at the time said that I need to si...


In [ ]:
#data cleaning
# lower case
# mutli-occurance of X removing
# removing digits and punctuations

In [27]:
import re

In [28]:
def text_cleaning(text):
  text = text.lower()   #lower case

  text = re.sub(r'[^\w\s]', '', text)  #removing punct

  text = re.sub(r'[0-9]', '', text)   #removing digits

  text = re.sub(r'[x]{2,}', '', text) # removing more than 2 occurance of X

  return text


In [29]:
sampled_df['Consumer complaint narrative'] = sampled_df['Consumer complaint narrative'].apply(text_cleaning)

In [30]:
sampled_df.head()

,Product,Consumer complaint narrative
0,loan,i signed a contract with equitable acceptance ...
1,loan,in i started the application process for the ...
2,loan,i went to pay on my load and as i was there i ...
3,loan,i have been enrolled in the public student loa...
4,loan,the company at the time said that i need to si...


In [31]:
sampled_df['Product'].unique()

array(['loan', 'credit_report', 'card', 'money_transfer', 'Mortgage',
       'Debt collection', 'account'], dtype=object)

In [32]:
classes = {'loan' : 0,
           'credit_report' :1,
           'card': 2,
           'money_transfer' : 3,
           'Mortgage' : 4,
           'Debt collection' : 5,
           'account' : 6
          }

sampled_df.replace({'Product' : classes}, inplace=True)

In [33]:
sampled_df

,Product,Consumer complaint narrative
0,0,i signed a contract with equitable acceptance ...
1,0,in i started the application process for the ...
2,0,i went to pay on my load and as i was there i ...
3,0,i have been enrolled in the public student loa...
4,0,the company at the time said that i need to si...
...,...,...
6995,6,my atm card s expiration date was when i did...
6996,6,i have a bank of america savings account i sub...
6997,6,i deposited a check on from a company that i ...
6998,6,we wrote a check for our rent in the amount of...


In [34]:
sampled_df.columns = ['label', 'text']

In [35]:

sampled_df.to_csv("complaint_dataset.csv", index=False)

In [52]:
# sampled_df.head(5)

In [36]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00


In [37]:
from datasets import load_dataset

In [ ]:
# data = load_dataset("fka/awesome-chatgpt-prompts")  --> download dataset from huggingface hub

In [ ]:
# data = load_dataset('csv', data_files=['csv1.csv', 'csv2.csv']) --> loading multiple csvs

In [ ]:
# data = load_dataset('csv', data_files={'train' : ['train1.csv', 'train2.csv']
#                                        'test' : ['test1.csv', 'test2.csv']})

In [ ]:
# from datasets import Dataset

# data = Dataset.from_pandas(df)  --> getting data from pandas

In [38]:
data = load_dataset('csv', data_files='complaint_dataset.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
data

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 7000
    })
})

In [40]:
train_test = data['train'].train_test_split(test_size=0.2, seed=42)

In [41]:
train_test

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5600
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1400
    })
})

In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [43]:
model_name = 'bert-base-cased'

In [44]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForSequenceClassification.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 7)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
def tokenize(batch):
  return tokenizer(batch['text'], truncation=True )

tokenized_data = train_test.map(tokenize, batched=True)

Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [46]:
from transformers import Trainer, TrainingArguments

In [47]:
training_args = TrainingArguments(output_dir = 'output_dir',
                                  evaluation_strategy = 'epoch',
                                  num_train_epochs= 2,
                                  )

In [48]:
from sklearn.metrics import f1_score

In [49]:
# Define evaluation metrics, which we will pass during training

def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = 'micro')
  return {'accuracy': acc, 'f1_score': f1}

In [50]:
tokenized_data['train']

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5600
})

In [51]:
trainer = Trainer(model,
                  training_args,
                  train_dataset = tokenized_data['train'],
                  eval_dataset = tokenized_data['test'],
                  tokenizer=tokenizer,
                  compute_metrics = compute_metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,1.137000,0.684884,0.786429,0.786429
2,0.667100,0.612495,0.820714,0.820714


TrainOutput(global_step=1400, training_loss=0.7957423727852958, metrics={'train_runtime': 1002.4739, 'train_samples_per_second': 11.172, 'train_steps_per_second': 1.397, 'total_flos': 2500658060446800.0, 'train_loss': 0.7957423727852958, 'epoch': 2.0})